In [1]:
import os
import numpy as np
import nltk
from nltk.tokenize import wordpunct_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.linear_model import LogisticRegression
import string
import pandas as pd
from gensim.models import Word2Vec

In [2]:
path_pos = "./data/pos/"
path_neg = "./data/neg/"

In [3]:
list_pos = os.listdir(path_pos)
list_neg = os.listdir(path_neg)

In [4]:
stemmer = PorterStemmer()

In [5]:
doc_pos=[]
doc_neg=[]

In [6]:
stop_words = stopwords.words("english")
stop_words.extend(list(string.punctuation))
stop_words = set(stop_words)

In [7]:
for p in list_pos:
    f = open(os.path.join(path_pos, p), 'r')
    lines = f.readlines()
    d=[]
    for l in lines:
        lw = wordpunct_tokenize(l)
        for w in lw:
            if w not in stop_words and not w.isnumeric():
                d.append(w)
    doc_pos.append(d)
    

In [8]:
for n in list_neg:
    f = open(os.path.join(path_neg, n), 'r')
    lines = f.readlines()
    d=[]
    for l in lines:
        lw = wordpunct_tokenize(l)    
        for w in lw:
            if w not in stop_words and not w.isnumeric():
                d.append(w)
    doc_neg.append(d)

In [9]:
(len(doc_neg),doc_neg[132]),(len(doc_pos),doc_pos[132])

((1000,
  ['showgirls',
   'first',
   'big',
   'budget',
   'big',
   'studio',
   'film',
   'receive',
   'nc',
   'rating',
   'release',
   'last',
   'year',
   'came',
   'time',
   'senator',
   'bob',
   'dole',
   'politicans',
   'chastised',
   'entertainment',
   'industry',
   'promoting',
   'sex',
   'violence',
   'showgirls',
   'indication',
   'hollywood',
   'future',
   'folks',
   'washington',
   'focus',
   'attack',
   'dangerous',
   'threat',
   'american',
   'values',
   '--',
   'bad',
   'moviemaking',
   'showgirls',
   'relentlessly',
   'exploitive',
   'look',
   'las',
   'vegas',
   'strip',
   'scene',
   'film',
   'turns',
   'trashy',
   'subject',
   'matter',
   'director',
   'paul',
   'verhoeven',
   'writer',
   'joe',
   'eszterhas',
   'collaborated',
   'another',
   'shock',
   'value',
   'project',
   'basic',
   'instinct',
   'follow',
   'plight',
   'young',
   'woman',
   'teen',
   'tv',
   'star',
   'elizabeth',
   'berkley

In [10]:
data = []
data = doc_pos.copy()
data.extend(doc_neg)

In [11]:
len(data)

2000

In [12]:
modelvec  = Word2Vec(sentences=data, min_count=1,vector_size= 128,workers=3, window =3, sg = 1)

In [13]:
modelvec.wv["feels"]

array([-0.04040681, -0.45635992, -0.02511956,  0.07583138,  0.24836947,
       -0.15505262,  0.01842343, -0.06884422, -0.01576013,  0.23680255,
        0.2041566 ,  0.02505621, -0.0932934 , -0.22280641,  0.11619628,
        0.15019295, -0.39979127,  0.2126427 , -0.32518098,  0.36726066,
        0.4686414 ,  0.14727013, -0.02481604, -0.44333825, -0.53436863,
        0.00156018, -0.25232425,  0.13516892, -0.10097696, -0.13606499,
       -0.15164618, -0.02054514,  0.16596733,  0.17308597, -0.31863084,
        0.21475114,  0.4134907 , -0.16443086,  0.01436855, -0.01670074,
       -0.10066611,  0.36434594, -0.24016018, -0.10304732,  0.03136501,
        0.01119312,  0.00166204,  0.05239561, -0.0925006 ,  0.07577536,
        0.3875662 ,  0.07049277,  0.2221499 ,  0.4351516 ,  0.114441  ,
        0.00844166,  0.380717  , -0.3206225 , -0.44363707, -0.02412039,
       -0.12190038, -0.09536015,  0.14970805, -0.10762133,  0.04885487,
       -0.31094205, -0.08134401,  0.09830882, -0.15366173, -0.18

******************************************

In [14]:
pos = []
neg = []

In [15]:
for d in doc_pos:
    p = np.zeros((128,))
    for w in d:
        p += modelvec.wv[w]
    pos.append(p/len(d))

In [16]:
for d in doc_neg:
    p = np.zeros((128,))
    for w in d:
        p += modelvec.wv[w]
    neg.append(p/len(d))

In [17]:
y_pos = np.ones((1000,))
y_neg = np.zeros((1000,))

In [18]:
X = np.concatenate([pos,neg],axis=0)
y = np.concatenate([y_pos,y_neg],axis=0)

X = (X - X.mean(axis=0))/(X.std(axis=0)+1e-7)

In [19]:
X.shape

(2000, 128)

In [20]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=43)

In [21]:
X_train.shape, y_train.shape    

((1600, 128), (1600,))

**********************************************

In [22]:
clf = LinearSVC(C=0.01)
clf.fit(X_train, y_train)

LinearSVC(C=0.01)

In [23]:
y = clf.predict(X_train)
np.sum((y == y_train))/y.shape[0]

0.770625

In [24]:
y = clf.predict(X_test)
np.sum((y == y_test))/y.shape[0]

0.78

***********************************

In [25]:
clf2 = LogisticRegression()
clf2.fit(X_train, y_train)

f:\aaaaaaaaaaa\Studying\4y2t\nlp\Labs\project\env\lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [26]:
y = clf2.predict(X_train)
np.sum((y == y_train))/y.shape[0]

0.790625

In [27]:
y = clf2.predict(X_test)
np.sum((y == y_test))/y.shape[0]

0.79